## Sample 1 - XGBoost 

I had the opportunity to start using xgboost machine learning algorithm, it is fast and shows good results. Here I will be using multiclass prediction with the iris dataset from scikit-learn.

http://ieva.rocks/2016/08/25/iris_dataset_and_xgboost_simple_tutorial/


In [5]:
import numpy as np

import xgboost as xgb

from sklearn import datasets

from sklearn.cross_validation import train_test_split

from sklearn.datasets import dump_svmlight_file

from sklearn.externals import joblib

from sklearn.metrics import precision_score, recall_score




iris = datasets.load_iris()

X = iris.data

y = iris.target



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# use DMatrix for xgbosot

dtrain = xgb.DMatrix(X_train, label=y_train)

dtest = xgb.DMatrix(X_test, label=y_test)



# use svmlight file for xgboost

dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)

dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)

dtrain_svm = xgb.DMatrix('dtrain.svm')

dtest_svm = xgb.DMatrix('dtest.svm')



# set xgboost params

param = {

    'max_depth': 3,  # the maximum depth of each tree

    'eta': 0.3,  # the training step for each iteration

    'silent': 1,  # logging mode - quiet

    'objective': 'multi:softprob',  # error evaluation for multiclass training

    'num_class': 3}  # the number of classes that exist in this datset

num_round = 20  # the number of training iterations



#------------- numpy array ------------------

# training and testing - numpy matrices

bst = xgb.train(param, dtrain, num_round)

preds = bst.predict(dtest)


# extracting most confident predictions

best_preds = np.asarray([np.argmax(line) for line in preds])

print("Numpy array precision:", precision_score(y_test, best_preds, average='macro'))
print("Numpy array recall:", recall_score(y_test, best_preds, average='macro'))




# ------------- svm file ---------------------

# training and testing - svm file

bst_svm = xgb.train(param, dtrain_svm, num_round)

preds = bst.predict(dtest_svm)



# extracting most confident predictions

best_preds_svm = [np.argmax(line) for line in preds]

print("Svm file precision:",precision_score(y_test, best_preds_svm, average='macro'))

# --------------------------------------------



# dump the models

bst.dump_model('dump.raw.txt')

bst_svm.dump_model('dump_svm.raw.txt')





# save the models for later

joblib.dump(bst, 'bst_model.pkl', compress=True)

joblib.dump(bst_svm, 'bst_svm_model.pkl', compress=True)

[05:21:27] 120x4 matrix with 480 entries loaded from dtrain.svm
[05:21:27] 30x4 matrix with 120 entries loaded from dtest.svm
Numpy array precision: 1.0
Numpy array recall: 1.0
Svm file precision: 1.0


['bst_svm_model.pkl']

## Sample 2 - XGBoost